# Hello Pyngres World

This is a minimal demonstration of the use of Pyngres. I connect to a database and select a table name at random from the iitables catalogue. When I am finished I tear everything down in an orderly fashion.

In addition to the **pyngres** module I will use the **ctypes** module in this demo. Everything in Python is an object so I use ctypes to marshal binary Ingres data to/from Python. Don't focus too much on ctypes; you probably won't use it in a real application.

In [ ]:
import pyngres as py
import ctypes

Before I can use the OpenAPI I have to initialize it. 

OpenAPI functions take a single argument which is a control block (structure) appropriate to the function being called. The initialization function expects an instance of the IIAPI_INITPARM structure, which is allocated by the pyngres **IIAPI_INITPARM()** function. Here I create an instance I name **inp**.

In [ ]:
inp = py.IIAPI_INITPARM()

Once I have allocated an OpenAPI control block I assign values to some of its members to control the function when I call it. Here I set **in_timeout** to -1 (meaning, "take as long as you like".) More importantly in this case, I also specify the version of the OpenAPI I want to use. Capabilities have been added over the years and if I depend on certain newer capabilities I can indicate what I need by setting **in_version**. Here I am setting it to **IIAPI_VERSION**, which is the latest version known to the current release of pyngres I have installed.

In [ ]:
inp.in_timeout = -1
inp.in_version = py.IIAPI_VERSION

I can now call **IIapi_initialize()**, passing its control block.

In [ ]:
py.IIapi_initialize(inp)

When a function returns, the control block is used to return information from the OpenAPI. In this case I am interested in the status of the call (which I don't check in this demonstration, but obviously you would). Assuming the call was successful the function returns **in_envHandle** which is a handle for the initialized environment. I need to supply that handle to other functions I will call, so I make a note of it.

In [ ]:
status = inp.in_status
envHandle = inp.in_envHandle

Before I can execute any queries I need to connect to Ingres, identifying the database I want to query. There is always a database called "iidbdb" so that is a convenient target database for this demonstration. Here I create a variable called **target**, initialized with the database name. But because the OpenAPI doesn't understand Python types I need to convert the Python str object 'iidbdb' into an array of bytes by calling the encode() method on it. We will see this technique several times in this short demonstration.

In [ ]:
target = 'iidbdb'.encode()

I create an IIAPI_CONNPARM control block and initialize it with the target name. I also pass the envHandle and set the connection type to IIAPI_CT_SQL (other connection types are available). I am willing to wait as long as it takes to get the connection so I set the timeout to -1.

In [ ]:
cop = py.IIAPI_CONNPARM()
cop.co_target = target
cop.co_connHandle = envHandle
cop.co_type = py.IIAPI_CT_SQL
cop.co_timeout = -1

With the connection parameters set up I execute **IIapi_connect()** to start a dialogue with the Ingres DBMS.

In [ ]:
py.IIapi_connect(cop)

Before I go on, we need to understand something very important about the OpenAPI. There are two kinds of functions: those that interact with the OpenAPI itself, and functions which interact with the Ingres DBMS. IIapi_initialize() is an example of the first kind. We just call those and they return in the usual way.

Most of the OpenAPI functions work differently. Because they are interacting with the DBMS—potentially over a channel with long latency—they are *asynchronous*. That means we call them but they return before the requested action is completed. While our requested OpenAPI action is in flight we are free to execute other code. **IIapi_connect()** is a typical OpenAPI function. It returns almost instantly but the connection request won't yet be completed.

Eventually we will need to check if a requested operation is finished. Usually we do that by running a busy-wait loop. (Callbacks are also possible.)

To run a wait loop I need an **IIAPI_WAITPARM** control block for it.

In [ ]:
wtp = py.IIAPI_WAITPARM()
wtp.wt_timeout = -1

I now sit in a busy-wait loop until the **gp_completed** flag is set in the IIAPI_CONNPARM control block (**cop**).

In [ ]:
while not cop.co_genParm.gp_completed:
    py.IIapi_wait(wtp)

Once the gp_completed flag is set I make a note of the connection handle because I will need it later. Incidentally, if I wanted I could now also check the error status. It would have been pointless trying to check it before the gp_completed flag is set. This timing issue sometimes perplexes the novice OpenAPI programmer.

In [ ]:
connHandle = cop.co_connHandle

Now I am ready to execute a query. Queries are written in SQL as you would expect, but Python strings mean nothing to Ingres so I need to encode the query to send it.

In [ ]:
# select a randomly chosen table name
queryText = 'SELECT FIRST 1 table_name FROM iitables ORDER BY random()'.encode()

This time I need an IIAPI_QUERYPARM control block. I initialize it with the connection handle so that Ingres knows which session the query should be sent to. I set the queryText attribute with my SQL statement. And I indicate this is a simple query by setting qy_queryType to IIAPI_QT_QUERY. (In the next demonstration I will show another type of query.)

In [ ]:
qyp = py.IIAPI_QUERYPARM()
qyp.qy_connHandle = connHandle
qyp.qy_queryType = py.IIAPI_QT_QUERY
qyp.qy_queryText = queryText

I can now submit my query by calling **IIapi_query**, which the OpenAPI will more or less instantly return, even though nothing may have happened yet. I need to sit in busy-wait (or do something else with my time, and then wait).

In [ ]:
py.IIapi_query(qyp)

while not qyp.qy_genParm.gp_completed:
    py.IIapi_wait(wtp)

This call initiated a strict protocol which has to be followed to get my results. Once the call is completed there is information available in the control block which I have to make note of to use later. Specifically the query could have implicitly initiated a transaction and I will need its handle. I will also need the statement handle to complete the protocol.

In [ ]:
tranHandle = qyp.qy_tranHandle
stmtHandle = qyp.qy_stmtHandle

To create a minimal example my query wasn't parameterized. If it were I would send the parameters at this point. In this example I can immediately fetch my results from the server. The first step is to get the meta-data for the result set using the **IIapi_getDescriptor()** function, which takes an **IIAPI_GETDESCPARM** control block. I set the gd_stmtHandle, call the function, then spin waiting for it to complete.

In [ ]:
gdp = py.IIAPI_GETDESCRPARM()
gdp.gd_stmtHandle = stmtHandle
py.IIapi_getDescriptor(gdp)
while not gdp.gd_genParm.gp_completed:
    py.IIapi_wait(wtp)

On completion the OpenAPI will return an array in the gd_descriptor member, describing each column of the result set with an **IIAPI_DESCRIPTOR** instance. I need to use those descriptors to allocate buffers for the returned columns. Obviously in this example I already know my query returns only one column but in general I should use the number of columns indicated by the gd_descriptorCount returned in the control block. 

To know how much memory to allocate for the buffer for a table name I look at the first element of the descriptor array and note the **ds_length**.

To fetch the data I will call **IIapi_getColumns()**, which takes an **IIAPI_GETCOLPARM**. I set the gc_rowCount to 1. That is not the expected number of rows but rather the number of rows the server is allowed to send me with one call to IIapi_getColumns(). I can ask for multiple rows if I want.

I use **ctypes.create_string_buffer()** to allocate storage for the table name, and I get its address using **ctypes.addressof()**. 

I then allocate the columnData array of **IIAPI_DATAVALUE** to pass to the OpenAPI. The syntax looks mysterious to a ctypes novice but don't dwell on it unless you plan to use ctypes. I set the dv_value of the first element of the array to the address of the table name storage. The array is passed using the gc_columnData member. I also have to set the gc_stmtHandle. 

With all that done I call IIapi_getColumns() and wait to hear back...eventually. 

In [ ]:
columnCount = gdp.gd_descriptorCount
length = gdp.gd_descriptor[0].ds_length

gcp = py.IIAPI_GETCOLPARM()
gcp.gc_rowCount = 1
gcp.gc_columnCount = columnCount
result_buffer = ctypes.create_string_buffer(length)
result_buffer_address = ctypes.addressof(result_buffer)
columnData = (py.IIAPI_DATAVALUE * (gcp.gc_rowCount * gcp.gc_columnCount))()
columnData[0].dv_value = result_buffer_address
gcp.gc_columnData = columnData
gcp.gc_stmtHandle = stmtHandle

py.IIapi_getColumns(gcp)
while not gcp.gc_genParm.gp_completed:
    py.Iapi_wait(wtp)

The chosen table name has now been sent from Ingres server to this application, but because Ingres sends binary data that Python doesn't understand I turn it into a Python string (str) using **ctypes.string_at()** and **decode()** so I can print it.

In [ ]:
table_name = ctypes.string_at(result_buffer_address,length).decode().strip()
print(f'OUTPUT--> the randomly chosen table name is: {table_name}')

I have my output so now I end the session. That involves ending the query statement; rolling back the transaction it implicitly started; disconnecting the session, and finally releasing the OpenAPI resources. You can see what aspect each OpenAPI call is acting on by looking at which handle it takes. 

In [ ]:
cnp = py.IIAPI_CANCELPARM()
cnp.cn_stmtHandle = stmtHandle
py.IIapi_cancel(cnp)
while not cnp.cn_genParm.gp_completed:
    py.IIapi_wait(wtp)

In [ ]:
clp = py.IIAPI_CLOSEPARM()
clp.cl_stmtHandle = stmtHandle
py.IIapi_close(clp)
while not clp.cl_genParm.gp_completed:
    py.IIapi_wait(wtp)

In [ ]:
rbp = py.IIAPI_ROLLBACKPARM()
rbp.rb_tranHandle = tranHandle
py.IIapi_rollback(rbp)
while not rbp.rb_genParm.gp_completed:
    py.IIapi_wait(wtp)
tranHandle = None

In [ ]:
dcp = py.IIAPI_DISCONNPARM()
dcp.dc_connHandle = connHandle
py.IIapi_disconnect(dcp)
while not dcp.dc_genParm.gp_completed:
    py.IIapi_wait(wtp)
connHandle = None

In [ ]:
rep = py.IIAPI_RELENVPARM()
rep.re_envHandle = envHandle
py.IIapi_releaseEnv(rep)

tmp = py.IIAPI_TERMPARM()
py.IIapi_terminate(tmp)

## The End 
(of the demonstration, not the world)